This notebooks reorganise by VOC species the monthly speciated NMVOCs for 2010 raw data from EDGARv4.3.2.

In [1]:
import xarray as xr
import numpy as np
import os

 ## Merge monthly files per species and sector

In [2]:
#data dir paths.
voc_dir='../edgarv5_emissions/NMVOC4.3.2/v432_VOC_spec/vocs_2010_monthly/'
save_dir='/geos/d21/s1878599/edgarv5_process/monthly_nmvocs4.3.2/'

In [3]:
# map between ipcc1996 (initial part of code) sector code and edgar_v4.3.2 sector code -
#( total secors number: 16. i.e. 17 - aviation supersonic (SPS) that has no monthly data. See https://edgar.jrc.ec.europa.eu/dataset_ap432_VOC_spec.
sec_map={'1A1':'ENE',
         '1A1b':'REF',
         '1A1c':'TRF',
         '1A2':'IND',
         '1A3a-CDS':'TNR-Aviation-CDS',
         '1A3a-CRS':'TNR-Aviation-CRS',
         '1A3a-LTO':'TNR-Aviation-LTO',
         '1A3b':'TRO',
         '1A3c':'TNR-Other',
         '1A3d':'TNR-Ship',
         '1A4':'RCO',
         '1B1a':'PRO',
         '2':'PPA',
         '4F':'AWB',
         '6':'SWD',
         '7A':'FFF'
        }

In [4]:
# GEIA/CEDS names for nmvoc used by edgar.
nmvoc=['voc1','voc2','voc3','voc4','voc5','voc6','voc7','voc8','voc9','voc10','voc11','voc12','voc13',
       'voc14','voc15','voc16','voc17','voc18','voc19','voc20','voc21','voc22','voc23','voc24','voc25']

In [5]:
# LOOP OVER NMVOC species

for voc in nmvoc:
    print(voc)
    for f in os.listdir(voc_dir): 
        if f !='total':
            voc_n= f.split('_')[3] # voc name.
            if voc_n==voc: #select only same voc among files.
                month=f.split('_')[5] # month.
                sector=f.split('_')[7].split('.')[0] #sector.
                if sector=='1A3a':
                    sector=sector+'-'+ f.split('_')[8].split('.')[0]  
    
                #add time coordinate and rename emission name.
                ds=xr.open_dataset(voc_dir+f)
                ds=ds.assign_coords({'time':int(month)})
                ds=ds.expand_dims(dim='time')
                ds=ds.rename_vars({('emi_'+voc_n):(sec_map[sector])}) 
    
                #temporalry monthly files.
                ds.to_netcdf(voc_n +'_' + month+'_'+ sec_map[sector] +'.nc',format='NETCDF3_64BIT')  

    #For a given species, put all sectors in a single file.     
    m_ds=xr.open_mfdataset('*.nc', combine='by_coords')
    
    # !add sectors that are missing (zero values). It will be easier to perform operation later with all sectors there.
    secs=list(m_ds.keys())
    for v in sec_map.values():
        if v not in secs:
            m_ds=m_ds.assign({v: xr.zeros_like(m_ds['ENE'])}) 
    
    #save
    m_ds.to_netcdf(save_dir +'/monthly_v432_2010_'+ voc +'_.0.1x0.1.nc') 
    
    ! rm *nc # remove temporary monthly files.

voc1
voc2
voc3
voc4
voc5
voc6
voc7
voc8
voc9
voc10
voc11
voc12
voc13
voc14
voc15
voc16
voc17
voc18
voc19
voc20
voc21
voc22
voc23
voc24
voc25


## Check

In [7]:
t= xr.open_dataset(save_dir+'monthly_v432_2010_voc3_.0.1x0.1.nc')

In [8]:
t

<xarray.Dataset>
Dimensions:           (lat: 1800, lon: 3600, time: 12)
Coordinates:
  * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon               (lon) float32 0.05 0.15 0.25 0.35 ... 359.75 359.85 359.95
  * time              (time) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    AWB               (time, lat, lon) float32 ...
    ENE               (time, lat, lon) float32 ...
    FFF               (time, lat, lon) float32 ...
    IND               (time, lat, lon) float32 ...
    PPA               (time, lat, lon) float32 ...
    PRO               (time, lat, lon) float32 ...
    RCO               (time, lat, lon) float32 ...
    REF               (time, lat, lon) float32 ...
    SWD               (time, lat, lon) float32 ...
    TNR-Aviation-CDS  (time, lat, lon) float32 ...
    TNR-Aviation-CRS  (time, lat, lon) float32 ...
    TNR-Aviation-LTO  (time, lat, lon) float32 ...
    TNR-Other         (time, lat, lon) float32 ...
    TNR-Ship          (time, lat, lon) float32 ...
    TRF               (time, lat, lon) float32 ...
    TRO               (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    title:             Monthly Mean (October) Emissions of VOC3 - 
    institution:       European Commission, Joint Research Centre
    source:            http://edgar.jrc.ec.europa.eu/
    history:           Created from original data (0.1x0.1 degrees) using IDL...
    references:        European Commission, Joint Research Centre (JRC)/Nethe...
    copyright_notice:  Reproduction of the data is authorized, except for com...
    contact:           edgar-info@jrc.ec.europa.eu